In [107]:
%reload_ext autoreload
%autoreload 2

import time
import numpy as np
from scipy import stats
import truncatedfa
from UTPPGF_util import *
from distributions import *

def new_utp(x, d):
    x_utp = UTPM(np.zeros((d, 1)))
    x_utp.data[0,0] = x
    if d > 1:
        x_utp.data[1,0] = 1
    return x_utp

def falling_factorial(k, i):
    return scipy.special.poch(i - k + 1, k)

def utp_deriv(x, k):
    dx = x.data.ravel()
    n = len(dx)
    fact = falling_factorial(k, np.arange(n))
    dx = dx * fact

    # shift by k places
    dy = np.zeros(n - k)
    dy[:n - k] = dx[k:]
    y = UTPM( dy.reshape(n-k, 1))
    return y

def utp_compose(G, F):
    # require G and F to have same dimension, and always output something of same size
    assert G.data.shape[0] == F.data.shape[0]

    g = G.data.copy().squeeze( axis=(1,) )
    f = F.data.copy().squeeze( axis=(1,) )
    g_scalar = g[0]
    g[0], f[0] = 0, 0
    
    d = len(g)
    
    # Horner's method truncated to d 
    res = np.array([g[d-1]])
    for i in range(d-2, -1, -1):
        res = np.convolve(res, f)[:d]
        res[0] += g[i]
        
    res[0] = g_scalar

    H = UTPM(res.reshape(-1,1))
    return H

# y = np.array([6,8,10,6,8,10,6,8,10])
y = np.array([100] * 9)
Lambda = np.array([16, 20, 24, 16, 20, 24, 16, 20, 24])
Delta = np.array([0.6, 0.4, 0.6, 0.4, 0.6, 0.4, 0.6, 0.4])
Rho = np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])

experiment = 'binomial'

if experiment == 'binomial':
    #poisson arrival, binom branching correctness
    arrival_pmf = stats.poisson
    arrival_pgf = lambda s, theta: poisson_pgf(s, theta)
    branch_fun  = truncatedfa.binomial_branching
    branch_pgf  = lambda s, theta: bernoulli_pgf(s, theta)
    observ_pgf  = None

    Theta = {'arrival': Lambda,
             'branch':  Delta,
             'observ':  Rho}
else:
    #poisson arrival, poisson branching correctness
    arrival_pmf = stats.poisson
    arrival_pgf = lambda s, theta: poisson_pgf(s, theta)
    branch_fun  = truncatedfa.poisson_branching
    branch_pgf  = lambda s, theta: poisson_pgf(s, theta)
    observ_pgf  = None

    Theta = {'arrival': Lambda,
             'branch':  Delta,
             'observ':  Rho}

K = len(y)

def lift_A(s, k, d):   # returns < A_k(s), ds >_d
    
    if k < 0:
        alpha = UTPM(np.zeros((d,1)))
        alpha.data[0,0] = 1.
        alpha.data[1,0] = 0.
        return alpha
    
#    print "k=%01d, y=%02d, d=%02d" % (k, y[k], d)

    F = lambda u: branch_pgf(u, Theta['branch'][k-1]) # branching PGF
    G = lambda u: arrival_pgf(u, Theta['arrival'][k])    # arrival PGF

    u = s * (1-Rho[k])
    s_prev = F(u)
    
    u_du = new_utp(u, d + y[k])
    beta = utp_compose( lift_A(s_prev, k-1, d+y[k]), F( u_du ) ) * G( u_du )

    s_ds = new_utp(s, d)
    alpha = utp_compose( utp_deriv(beta, y[k]), s_ds*(1-Rho[k]) ) / scipy.misc.factorial(y[k]) * np.power( s_ds * Rho[k], y[k])
    
    return alpha

tic = time.time()
alpha = lift_A(1, K-1, 1) 
print alpha.data[0,0]

toc = time.time()
print "%.4f seconds" % (toc - tic)

4.52142777843e-119
3.8297 seconds
